In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving energydata_complete.csv to energydata_complete.csv
User uploaded file "energydata_complete.csv" with length 11979363 bytes


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset Description

**The dataset for the remainder of this quiz is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. 
The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. 
Each wireless node transmitted the temperature and humidity conditions around 3.3 min. 
Then, the wireless data was averaged for 10 minutes periods. 
The energy data was logged every 10 minutes with m-bus energy meters. 
Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru),
 and merged together with the experimental data sets using the date and time column.
 Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). 
The attribute information can be seen below.**

Attribute Information:

- Date, time year-month-day hour:minute:second

- Appliances, energy use in Wh

- lights, energy use of light fixtures in the house in Wh

- T1, Temperature in kitchen area, in Celsius

- RH_1, Humidity in kitchen area, in %

- T2, Temperature in living room area, in Celsius

- RH_2, Humidity in living room area, in %

- T3, Temperature in laundry room area

- RH_3, Humidity in laundry room area, in %

- T4, Temperature in office room, in Celsius

- RH_4, Humidity in office room, in %

- T5, Temperature in bathroom, in Celsius

- RH_5, Humidity in bathroom, in %

- T6, Temperature outside the building (north side), in Celsius

- RH_6, Humidity outside the building (north side), in %

- T7, Temperature in ironing room , in Celsius

- RH_7, Humidity in ironing room, in %

- T8, Temperature in teenager room 2, in Celsius

- RH_8, Humidity in teenager room 2, in %

- T9, Temperature in parents room, in Celsius

- RH_9, Humidity in parents room, in %

- To, Temperature outside (from Chievres weather station), in Celsius

- Pressure (from Chievres weather station), in mm Hg

- RH_out, Humidity outside (from Chievres weather station), in %

- Wind speed (from Chievres weather station), in m/s

- Visibility (from Chievres weather station), in km

- Tdewpoint (from Chievres weather station), Â °C

- rv1, Random variable 1, nondimensional

- rv2, Random variable 2, nondimensional

***To answer some questions, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). 
Run a multiple linear regression using the training set and evaluate your model on the test set.***


In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
# Load Dataset
energy = pd.read_csv('/content/energydata_complete.csv')

energy.head(5)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [60]:
# Define our predictors and target variable
df = energy.copy()


# df = df[df['Windspeed'] > 0]
y = df[['Appliances']]
X = df.drop(['date', 'Appliances', 'lights'], axis=1)

In [61]:
# my_list = ['T2', 'RH_5', 'RH_out', 'Windspeed']

# from sklearn.preprocessing import PowerTransformer

# pt = PowerTransformer()

# mean = X['Windspeed'].mean()
# std = X['Windspeed'].std()
# cut_off = std * 3
# lower, upper = mean - cut_off, mean + cut_off
# mask = (X.Windspeed < upper) & (X.Windspeed > lower)


# norm = X[mask]

# transd = pd.DataFrame(pt.fit_transform(norm[['Windspeed']]), columns=['Windspeed'])


In [62]:


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



X_train, X_test, y_train, y_test = train_test_split(X[['T2']], X['T6'], test_size=.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
RSS = np.sum(np.square(y_test - y_pred))
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

print('R2', round(r2, 2))
print('MAE', round(MAE, 2))
print('RSS', round(RSS, 2))
print('RMSE', round(RMSE, 3))

R2 0.64
MAE 2.82
RSS 78033.97
RMSE 3.63


In [63]:
#  Summary Statistics
print(X.shape)
print(y.shape)

X.describe()

(19735, 26)
(19735, 1)


,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,19.592106,50.949283,7.910939,54.609083,20.267106,35.388200,22.029107,42.936165,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,1.844623,9.022034,6.090347,31.149806,2.109993,5.114208,1.956162,5.224361,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,15.330000,29.815000,-6.065000,1.000000,15.390000,23.200000,16.306667,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,18.277500,45.400000,3.626667,30.025000,18.700000,31.500000,20.790000,39.066667,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,19.390000,49.090000,7.300000,55.290000,20.033333,34.863333,22.100000,42.375000,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,20.619643,53.663333,11.256000,83.226667,21.600000,39.000000,23.390000,46.536000,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,25.795000,96.321667,28.290000,99.900000,26.000000,51.400000,27.230000,58.780000,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [64]:
my_list = ['T2', 'RH_5', 'RH_out', 'Windspeed']
# my_list = [col for col in X.columns if col not in ['T6', 'T_out', 'Tdewpoint']]

def box_trans(my_list):
  print('Performing Boxcox transformation.....')
  from scipy.stats import boxcox

  for var in my_list:
    X[var], lambd = boxcox(X[var])





In [65]:
def fit_score(model, X_train, y_train, X_test, y_test):
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  train_score = model.score(X_train, y_train)
  cv_mean = np.mean(cross_val_score(model, X_train, y_train, cv=5))
  rmse = np.sqrt(mean_squared_error(y_test, y_pred)) #Root Mean Squared Error
  r2 = r2_score(y_test, y_pred)

  print('Train Score:\t', round(train_score, 3))
  print('Cross Validation Mean Score:\t', round(cv_mean, 3))
  print('RMSE:\t', round(rmse, 3))
  print('R2 score:\t', round(r2, 3))

In [66]:
def poly(X, Xt, poly=False, deg=1):
  from sklearn.preprocessing import PolynomialFeatures

  if poly is True:
    pf = PolynomialFeatures(degree=deg, include_bias=False)
    X = pf.fit_transform(X)
    Xt = pf.transform(Xt)

  return X, Xt

In [67]:
def scale(X_train, X_test):
  from sklearn.preprocessing import MinMaxScaler

  scaler = MinMaxScaler()
  X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
  X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

  return X_train, X_test

In [68]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# Apply MinMaxScaler
X_train, X_test = scale(X_train, X_test)

In [69]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
RSS = np.sum(np.square(y_test - y_pred))
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

print('R2', round(r2, 2))
print('MAE', round(MAE, 2))
print('RSS', round(RSS, 2))
print('RMSE', round(RMSE, 3))

R2 0.15
MAE 53.64
RSS Appliances    51918501.21
dtype: float64
RMSE 93.64


In [70]:
ridge = Ridge(alpha=0.4)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_test)
r2 = r2_score(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
RSS = np.sum(np.square(y_test - y_pred))
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

print('R2', round(r2, 2))
print('MAE', round(MAE, 2))
print('RSS', round(RSS, 2))
print('RMSE', round(RMSE, 3))

R2 0.15
MAE 53.59
RSS Appliances    51940210.67
dtype: float64
RMSE 93.66


In [71]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)
r2 = r2_score(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
# RSS = np.sum(np.square(y_test - y_pred))
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

print('R2', round(r2, 2))
print('MAE', round(MAE, 2))
# print('RSS', round(RSS, 2))
print('RMSE', round(RMSE, 3))

R2 0.15
MAE 53.63
RMSE 93.641


In [72]:
pd.DataFrame(lasso.coef_)

,0
0,-4.055859
1,494.326194
2,-249.638653
3,-465.821050
4,310.200156
5,102.172472
6,30.344205
7,26.560581
8,-17.113012
9,17.013556


In [73]:
pd.DataFrame(lr.coef_, columns=X.columns).T.sort_values(0, ascending=False)

,0
RH_1,495.526020
T3,310.971037
T6,252.716928
Tdewpoint,125.430637
T8,109.134698
RH_3,102.771652
RH_6,40.712058
Windspeed,31.225948
T4,31.009668
RH_4,27.311829
